In [1]:
!pip install mysqlclient

You should consider upgrading via the 'c:\users\administrator\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import MySQLdb

In [3]:
#DTO 구성

class DTOA:
    def __init__(self, number, name, hp, email, address):
        self.number = number
        self.name = name
        self.hp = hp
        self.email =email
        self.address =address
    
    def setNumber(self, number):
        self.number = number
    
    def getNumber(self):
        return self.number

    def setName(self, name):
        self.name = name
    
    def getName(self):
        return self.name

    def setHp(self, hp):
        self.hp = hp
    
    def getHp(self):
        return self.hp
   
    def setEmail(self, email):
        self.email = email
    
    def getEmail(self):
        return self.email
    
    def setAddress(self, address):
        self.address = address
    
    def getAddress(self):
        return self.address

In [4]:
#DTO 구성2

class DTOB:
    def __init__(self, number, javagrade, pythongrade, cgrade,total,avg):
        self.number = number
        self.javagrade = javagrade
        self.pythongrade = pythongrade
        self.cgrade =cgrade
        self.total = total
        self.avg   = avg

            
    def setNumber(self, number):
        self.number = number
    
    def getNumber(self):
        return self.number

    def setJavagrade(self, javagrade):
        self.javagrade = javagrade
    
    def getJavagrade(self):
        return self.javagrade

    def setPythongrade(self, pythongrade):
        self.pythongrade = pythongrade
    
    def getPythongrade(self):
        return self.pythongrade
   
    def setCgrade(self, cgrade):
        self.cgrade = cgrade
    
    def getCgrade(self):
        return self.cgrade
    
    def setTotal(self,total):
        self.total=total
        
    def getTotal(self):
        return self.total
    
    def setAvg(self,avg):
        self.avg =avg
    
    def getAvg(self):
        return self.avg

In [5]:
#DAO 구성

class DAO:
    def __init__(self):
        self.db = None
    #처리시 연결담당
    def connect(self):
        self.db = MySQLdb.connect('localhost', 'root', '1234', 'test')
    #처리후 연결끊기    
    def disconnect(self):
        self.db.close()
    
    #data를 입력한다.
    def insert(self, it):
        self.connect()
        cur = self.db.cursor()
        sql ='insert into student values(%s,%s,%s,%s,%s,now())'
        data = (it.getNumber(), it.getName(), it.getHp(), it.getEmail(),it.getAddress())
        cur.execute(sql, data)
        self.db.commit()
        self.disconnect()     
        
    def insert2(self, it):
        self.connect()
        cur = self.db.cursor()
        sql ='insert into grade values(%s,%s,%s,%s,now(),%s,%s)'
        data = (it.getNumber(), it.getJavagrade(), it.getPythongrade(), it.getCgrade(), it.getTotal(), it.getAvg())
        cur.execute(sql, data)
        self.db.commit()
        self.disconnect()
        
    def selectSt(self):
        self.connect()
        cur = self.db.cursor()
        sql = 'select number,name,hp,email,address,regdate from student'
        cur.execute(sql)
        row = cur.fetchall()
        self.disconnect()
        return row
    
        # student 과 grade table을 뷰로써 left join 하였다. 반복 되며  sql오류가 나면 안되기에 create or replace를 사용하였다.
    def selectPr(self):
        self.connect()
        cur = self.db.cursor()
        sql1 = 'create or replace view vw_Pr8 as select s.number, s.name , s.hp, s.email,s.address,s.regdate, g.javagrade,g.pythongrade,g.cgrade,g.avg,g.total from student as s left join grade as g on s.number = g.number order by g.avg desc,s.number desc'
        cur.execute(sql1)
        sql2 = 'select number,name,hp,email,address,regdate,javagrade,pythongrade,cgrade,avg,total from vw_pr8'
        #뷰테이블로 데이터를 가져왔다.
        cur.execute(sql2)
        row = cur.fetchall()
        self.disconnect()
        return row

    #select를 통하여 row값을 리턴해준다.
    def searchSt(self, number):
        self.connect()
        cur = self.db.cursor()
        sql = 'select  number,name,hp,email,address,regdate from student where number=%s'
        data = (number,)     
        cur.execute(sql, data)
        row = cur.fetchone()
        self.disconnect()
        return row
    
    def searchGr(self, number):
        self.connect()
        cur = self.db.cursor()
        sql = 'select  number,javagrade,pythongrade,cgrade from grade where number=%s'
        data = (number,)     
        cur.execute(sql, data)
        row = cur.fetchone()
        self.disconnect()
        return row

    #update 문을 사용하여 원래 값을 수정해준다.
    def updateSt(self, it):
        self.connect()
        cur = self.db.cursor()
        sql = 'update student set name=%s, hp=%s, email=%s, address=%s,regdate=now() where number=%s'
        data = (it.getName(), it.getHp(), it.getEmail(),it.getAddress(),it.getNumber())
        result = cur.execute(sql, data)
        self.db.commit()
        #sql문이 정상적으로 수행되었을때 1이 나오므로 if문을 사용하여 사용자에게 메시지를 전달한다.
        if result > 0:
            print('수정되었습니다')
        else:
            print('해당 학생이 없습니다')
        self.disconnect()

        
    def updateGr(self, it):
        self.connect()
        cur = self.db.cursor()
        sql = 'update grade set javagrade=%s,pythongrade=%s, cgrade=%s,regdate=now(),total=%s,avg=%s where number=%s'
        data = (it.getJavagrade(), it.getPythongrade(), it.getCgrade(),it.getTotal(),it.getAvg(),it.getNumber())
        result = cur.execute(sql, data)
        self.db.commit()
        if result > 0:
            print('수정되었습니다')
        else:
            print('해당 학생이 없습니다')
        self.disconnect()        
        
    #delete 문으로 정보를 삭제한다. 
    #sql 파일안에 
    #ALTER TABLE grade
    #ADD CONSTRAINT FK_B_1 FOREIGN KEY (number)
    #REFERENCES student (number) ON DELETE CASCADE; 문을 사용하여 부모테이블이 삭제되면 자식테이블의 정보도 삭제되게 설정하여준다.
    def deleteSt(self, number):
        self.connect()
        cur = self.db.cursor()
        sql = 'delete from student where number=%s'
        data = (number,)
        result = cur.execute(sql, data)
        self.db.commit()
        if result > 0:
            print('삭제되었습니다')
        else:
            print('해당 학생이 없습니다')
        self.disconnect()
        
        
    def deleteGr(self, number):
        self.connect()
        cur = self.db.cursor()
        sql = 'delete from grade where number=%s'
        data = (number,)
        result = cur.execute(sql, data)
        self.db.commit()
        if result > 0:
            print('삭제되었습니다')
        else:
            print('해당 학생이 없습니다')
        self.disconnect()

In [6]:
#data를 받아 DTO에 전달한후 DAO로 정보를 처리한다.

class Service:
    def __init__(self):
        self.dao = DAO()
    def insertData(self):
        number = input("학번을 입력하세요:")
        name = input('이름를 입력하세요:')
        hp = input('연락처를 입력하세요:')
        email = input('이메일을 입력하세요:')
        address = input('주소를 입력하세요:')            
        A = DTOA(number,name, hp, email, address)
        self.dao.insert(A)
        
    def insertData2(self):
        number = input("학번을 입력하세요:")
        javagrade = input('자바성적을 입력하세요:')
        pythongrade = input('파이썬 성적을 입력하세요:')
        cgrade = input('C언어 성적을 입력하세요:')   
        total = int(javagrade)+int(pythongrade)+int(cgrade)
        avg   = (total)/3
        A = DTOB(number,javagrade,pythongrade,cgrade,total,avg)
        self.dao.insert2(A)    
  
    #나오는 data값을 인덱싱을 통하여 보기 좋게 다듬었다.  try문을 이용하여 출력값이 없을 경우를 처리해주었다.
    def printAll(self):
        try:
            A=datas = self.dao.selectPr()
            for i in range(len(A)):
                print("학번:",A[i][0])
                print("이름:",A[i][1])
                print("연락처:",A[i][2])
                print("이메일:",A[i][3])
                print("주소:",A[i][4])
                print("등록된 시간:",A[i][5])
                print("java점수:",A[i][6])
                print("파이썬점수:",A[i][7])
                print('C언어점수:',A[i][8])
                print('평균:',A[i][9])
                print('총점:',A[i][10])
                print("석차:",i+1)    
                print("총원:",len(A))
                print("\n")
                
        except:
            print("출력할 정보가 없습니다.")
        
    #if문을 사용하여 점수가 있을경우와 없을 경우를 나누어서 검색후 출력해주었다.+
    def searchData(self):
        number = input('검색할 사람의 학번을 입력하세요')
        it = self.dao.searchSt(number)
        B = self.dao.searchGr(number)
        if B == None:     
            if it:
                print("학번:",it[0])
                print("이름:",it[1])
                print("연락처:",it[2])
                print("이메일:",it[3])
                print("주소:",it[4])
            else:
                print('해당 학생이 없습니다') 
        else:
            print("학번:",it[0])
            print("이름:",it[1])
            print("연락처:",it[2])
            print("이메일:",it[3])
            print("주소:",it[4])
            print("자바점수:",B[1])
            print("파이썬점수:",B[2])
            print("C언어 점수:",B[3])
            
            
            
    def editData(self):
        number = input('수정할 사람의 학번을 입력하세요')
        it = self.dao.searchSt(number)
        if it == None:
            print('수정할 사람을 찾지 못했습니다')
        else:
            name = input('이름를 입력하세요:')
            hp = input('연락처를 입력하세요:')
            email = input('이메일을 입력하세요:')
            address = input('주소를 입력하세요:')
            A = DTOA(number, name, hp, email, address)
            it = self.dao.updateSt(A)
            
        
    def editData2(self):
        number = input('수정할 사람의 학번을 입력하세요')
        it = self.dao.searchGr(number)
        if it == None:
            print('수정할 사람을 찾지 못했습니다')
        else:
            javagrade = input('자바성적을 입력하세요:')
            pythongrade = input('파이썬 성적을 입력하세요:')
            cgrade = input('C언어 성적을 입력하세요:')   
            total = int(javagrade)+int(pythongrade)+int(cgrade)
            avg   = (total)/3
            
            New = DTOB(number,javagrade,pythongrade,cgrade,total,avg)
            it = self.dao.updateGr(New)
        
    def delData(self):
        number = input('삭제할 사람의 학번을 입력하세요')
        it = self.dao.deleteSt(number)
        
    def delData2(self):
        number= input('성적을 삭제할 사람의 학번을 입력하세요')
        it = self.dao.deleteGr(number)
        

In [7]:
class Menu:
    def __init__(self):
        self.service = Service()
    
    def run(self):
        while True:
            try:
                print('\n학생관리 프로그램\n')
                menu = int(input(
                             '1.학생 정보 입력\n'
                             '2.학생 정보 출력\n' 
                             '3.학생 정보 검색\n' 
                             '4.학생 정보 수정\n' 
                             '5.학생 정보 삭제\n' 
                             '6.학생 점수 등록\n' 
                             '7.학생 점수 수정\n'
                             '8.학생 점수 삭제\n'
                             '9.프로그램 종료\n' ))

                if menu == 1:
                    self.service.insertData()
                elif menu == 2:
                    self.service.printAll()
                elif menu == 3:
                    self.service.searchData()
                elif menu == 4:
                    self.service.editData()
                elif menu == 5:
                    self.service.delData()
                elif menu == 6:
                    self.service.insertData2()
                elif menu == 7:
                    self.service.editData2()
                elif menu == 8:
                    self.service.delData2()
                elif menu == 9:
                    break
            except Exception as e:
                print(e)
                print('다시 입력하세요')


In [ ]:
start =Menu()
start.run()


학생관리 프로그램

1.학생 정보 입력
2.학생 정보 출력
3.학생 정보 검색
4.학생 정보 수정
5.학생 정보 삭제
6.학생 점수 등록
7.학생 점수 수정
8.학생 점수 삭제
9.프로그램 종료
2
학번: 2
이름: 2번
연락처: 011
이메일: 2@naver.com
주소: 인천
등록된 시간: 2023-03-31 15:45:58
java점수: 50
파이썬점수: 40
C언어점수: 10
평균: 33.333333333333336
총점: 100
석차: 1
총원: 1



학생관리 프로그램

1.학생 정보 입력
2.학생 정보 출력
3.학생 정보 검색
4.학생 정보 수정
5.학생 정보 삭제
6.학생 점수 등록
7.학생 점수 수정
8.학생 점수 삭제
9.프로그램 종료
2
학번: 2
이름: 2번
연락처: 011
이메일: 2@naver.com
주소: 인천
등록된 시간: 2023-03-31 15:45:58
java점수: 50
파이썬점수: 40
C언어점수: 10
평균: 33.333333333333336
총점: 100
석차: 1
총원: 1



학생관리 프로그램

1.학생 정보 입력
2.학생 정보 출력
3.학생 정보 검색
4.학생 정보 수정
5.학생 정보 삭제
6.학생 점수 등록
7.학생 점수 수정
8.학생 점수 삭제
9.프로그램 종료
4
수정할 사람의 학번을 입력하세요1
수정할 사람을 찾지 못했습니다

학생관리 프로그램

1.학생 정보 입력
2.학생 정보 출력
3.학생 정보 검색
4.학생 정보 수정
5.학생 정보 삭제
6.학생 점수 등록
7.학생 점수 수정
8.학생 점수 삭제
9.프로그램 종료
6
학번을 입력하세요:2
자바성적을 입력하세요:20
파이썬 성적을 입력하세요:20
C언어 성적을 입력하세요:30

학생관리 프로그램

1.학생 정보 입력
2.학생 정보 출력
3.학생 정보 검색
4.학생 정보 수정
5.학생 정보 삭제
6.학생 점수 등록
7.학생 점수 수정
8.학생 점수 삭제
9.프로그램 종료
2
학번: 2
이름: 2번
연락처: 011
이메일: 2@naver.com
주소: 